In [1]:
import os
from selenium import webdriver
import time
import io
from PIL import Image
import hashlib
import pandas as pd

import errno
from functools import wraps
import signal

In [2]:
# Constants

# Replace with your driver path!
# download available from https://sites.google.com/a/chromium.org/chromedriver/downloads
DRIVER_PATH = '../chromedriver_linux64/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

data_folder="../data"

In [3]:
product_df = pd.read_csv(os.path.join(data_folder, "products.csv"))
product_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [25]:
class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

In [32]:
@timeout(30)
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        #print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            #print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [13]:
@timeout(2)
def persist_image(folder_path:str,url:str,file_name:str):
    print("persistent image")
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        return False
    
    print("downloaded")
    
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,file_name + '.jpg')
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
        return False
    return True

In [14]:
@timeout(5)
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=3):
    print("search and download")
    target_folder = os.path.join(target_path)#,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    if res is None:
        print("No links found :(")
        return
    
    for elem in res:
        try:
            if persist_image(target_folder,elem, search_term):
                break
        except TimeoutError:
            print("problem with persist_image, skipping...")
            
    print("done")

In [15]:
# Test
search_term = "dog"
search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

search and download


TimeoutError: Timer expired

In [33]:
link_list = []

# Download an image of each product
with webdriver.Chrome(executable_path=DRIVER_PATH) as wd:
    for search_term in product_df.product_name[17894:]:
        try:
            link_list.append(fetch_image_urls(query=search_term, max_links_to_fetch=1, wd=wd).pop())
        except AttributeError:
            link_list.append("https://cdn-a.william-reed.com/var/wrbm_gb_food_pharma/storage/images/9/2/8/5/235829-6-eng-GB/Feed-Test-SIC-Feed-20142_news_large.jpg")
        except Exception:
            link_list.append("https://cdn-a.william-reed.com/var/wrbm_gb_food_pharma/storage/images/9/2/8/5/235829-6-eng-GB/Feed-Test-SIC-Feed-20142_news_large.jpg")

print(link_list)

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

...

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 2 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 2 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 2 image links

Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 1 image links, done!
Found: 2 image links, done!
Found: 1 image links, done!
Found: 1 image links

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
print(len(link_list))
#17894?

31794


In [35]:
with open("link_list_31794.txt",'w') as ll:
    ll.writelines(link_list)